### 二. 指针与结构体
1. 指针  
 指针保存了变量所在的内存地址
   1. `&` : 生成一个指向变量的指针
   2. 指针类型`*T` : 称作`T的指针类型`. `*`代表指针. 
     ```go
     aaa := 1
     bbb := "aaa"
     fmt.Printf("%p %p",&aaa,&bbb)
      ```
   3. 当使用`&`操作符对普通变量进行取地址操作并得到变量的指针后，可以对指针使用`*`操作符，也就是指针取值. 
        ```go
        str := "hello world"
        ptr := &str

        fmt.Printf("ptr type: %T\n", ptr) // 打印ptr的类型
        fmt.Printf("address: %p\n", ptr)  // 打印ptr指向的地址,

        value := *ptr  // 对指针取值
        fmt.Printf("value type: %T\n", value)
        fmt.Printf("value: %s\n", value)
        
        // ptr type: *string
        // address: 0xc0420461c0
        // value type: string
        // value: hello world
        ```
   4. 通过指针修改值  
     `*` 操作符的根本意义就是: 操作指针指向的变量。当出现在等号右边时，就是取指向变量的值，当出现在等号左边时，就是将值设置给指向的变量。
        ```go
        func swap(a,b *int){
            t := *a
            *a = *b
            *b = t
        }

        func main(){
            x,y := 1,2
            swap(&x,&y)
            fmt.Println(x,y)
        }

        ```
       如果在`swap()`函数中交换操作的是指针值，会发生什么情况？可以参考下面代码：
        ```go
        func swap(a,b *int){
            a,b = b,a
        }

        func main(){
            x,y := 1,2
            swap(&x,&y)
            fmt.Println(x,y)  // 1,2
        }
        ```
        结果表明，交换是不成功的。上面代码中的 swap() 函数交换的是 a 和 b 的地址，在交换完毕后，a 和 b 的变量值确实被交换。但和 a、b 关联的两个变量并没有实际关联。这就像写有两座房子的卡片放在桌上一字摊开，交换两座房子的卡片后并不会对两座房子有任何影响。

   5. 创建指针的另一种方法——new() 函数  
        * Go语言还提供了另外一种方法来创建指针变量，格式如下：`new(类型)`
        ```go
        str := new(string)
        *str = "go go go"
        fmt.Println(*str)
        ```
        * `new(type)`创建指针的方式, 可以避免由于修改一个空指针的指向而编译出错, 如下代码揭示了这一现象
        ```go
        var str2 *string  
        *str2 = "gogogo"  // panic: runtime error: invalid memory address or nil pointer dereference
        fmt.Println(*str2)
        ```
        这个问题出现的原因是, 指针类型的数据起初始值是`nil`, 而`nil`没有具体指向, 也就不能更改器指向. 要想修复这问题, 就要使用上一个代码片段中的`new(type)`进行声明

   6. 方法中的指针  
        方法即为有"receiver"的函数. 这里的"reciever"指调用该函数的结构体实例. "reciever"既可以是实例变量也可以是指针类型的变量. 但二者在调用效果上存在差别:   
      1. `value method`: "reciever"为实例的方法   
           * 符号: `func (T) method(param) rtype{}`
           * 值方法既可以被实例调用也可以被指针调用, 但是这种方式即使在方法内部有修改reciever值的代码, 也不能在函数退出后永久性的修改reciever
            ```go
            type Person struct{
                name string
                age int
            }

            func(p Person) say(){
                fmt.Println(p.name,":",p.age)
            }

            func (p Person) changeAge(newAge int){
                p.age = newAge
            }

            func main()  {
                p := Person{"zhangsan",23}
                (&p).changeAge(28)  // 指针调用值方法
                p.say()  // 23
                
                p.changeAge(28)     // 实例调用值方法
                p.say()  // 23
            }
            ```
      2. `pointer methd`: "reciever"为指针的方法
           * 符号: `func (*T) method(param) rtype{}`
           * pointer method被指针类型的变量调用, 因为pointer method的本质是要更改reciever的值, 如果用value类型的变量调用就会导致值拷贝而无法该值, 这会造成语义上的冲突. 为了解决这个问题, 编译器如果发现是一个value型变量调用了`pointer method`, 编译器会自动在前面加上`&`, 改写成`(&value)`. 如下代码所示
            ```go
            type Person struct{
                name string
                age int
            }

            func(p Person) say(){
                fmt.Println(p.name,":",p.age)
            }

            func (p *Person) changeAge(newAge int){
                p.age = newAge
            }

            func main()  {
                p := Person{"zhangsan",23}
                (&p).changeAge(28)  // 指针调用指针方法
                p.say()  // 28

                p.changeAge(29)     // 实例调用指针方法
                p.say()  // 29, 编译器会自动将该调用改写成上面的(&p).changeAge(28)
            }
            ```

  
2. 结构体  
    1. 结构体是一组字段, 用`type [struct name] struct`来声明
    2. 使用`.`访问结构体的某一个字段
    3. 用`TypeName{Fileld1 \n Field2}`来创造结构体对象
        ```go
        type Vertex struct{
            X int
            Y int
        }
        ```
    4. 可以只对结构体的某几个字段赋值, 其他几个字段使用结构体字段类型的默认值. `structname{Field1: value}`
